Para utilizar o PyTorch, primeiro devemos instalá-lo:

In [ ]:
python
# Instalação do PyTorch
!pip install torch torchvision torchaudio

# Importando o PyTorch
import torch
import torch.nn as nn
import torch.optim as optim


O PyTorch fornece tensors que são estruturas de dados multidimensionais similares aos arrays do NumPy, além de operações diferenciáveis para construção de redes neurais.

Para carregar os dados, podemos utilizar as mesmas bibliotecas como Pandas e NumPy:

In [ ]:
python
import pandas as pd
import numpy as np

# Lendo dados de treino e teste
X_treino = pd.read_csv("treino.csv")
X_teste = pd.read_csv("teste.csv")

# Convertendo para Tensors
X_treino = torch.from_numpy(X_treino.values)
X_teste = torch.from_numpy(X_teste.values)


O pré-processamento também segue um fluxo similar, porém agora utilizando a biblioteca torch ao invés de sklearn:

In [ ]:
python
# Normalizando os dados
transforms = torch.nn.Sequential(
    torch.nn.Linear(num_features, num_features),
    torch.nn.BatchNorm1d(num_features),
    torch.nn.ReLU(),
)

X_treino = transforms(X_treino)
X_teste = transforms(X_teste)




Para definir e treinar a rede neural, utilizamos as classes nn.Module e nn.Sequential do PyTorch:

In [ ]:
python
# Definindo a CNN
modelo = nn.Sequential(
    nn.Conv2d(...),
    nn.MaxPool2d(...),
    nn.Flatten(),
    nn.Linear(...)
)

# Definindo a função de loss e otimizador
criterio = nn.CrossEntropyLoss()
otimizador = torch.optim.Adam(modelo.parameters(), lr=0.001)

# Treinando a rede
for epoca in range(10):
    y_pred = modelo(X_treino)
    loss = criterio(y_pred, y_treino)
    loss.backward()
    otimizador.step()
    otimizador.zero_grad()

# Avaliando no conjunto de teste
with torch.no_grad():
    y_test_pred = modelo(X_teste)
    acc = (y_test_pred.argmax(dim=1) == y_teste).float().mean()


Como se nota, utilizamos o PyTorch para treinar redes neurais convolucionais.

Além da instalação e importação das bibliotecas, definimos o device CPU, no qual o modelo e os dados serão alocados:

In [ ]:
python
# Definindo o device
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

# Modelo para o device
modelo = modelo.to(device)

# Dados para o device
X_treino = X_treino.to(device)
y_treino = y_treino.to(device)
X_teste = X_teste.to(device)
y_teste = y_teste.to(device)


Outra técnica importante é o uso de DataLoaders e DataSets para carregar os dados em batches:

In [ ]:
python
# Imports
from torch.utils.data import Dataset, DataLoader

# Dataset
class MeuDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)

# DataLoaders
train_loader = DataLoader(MeuDataset(X_treino, y_treino), batch_size=32)
test_loader = DataLoader(MeuDataset(X_teste, y_teste), batch_size=32)


Sendo assim, o treinamento fica:

In [ ]:
python
for epoca in range(10):

  for batch in train_loader:

    X, y = batch

    # Treinamento

    loss.backward()
    otimizador.step()
    otimizador.zero_grad()

  # Validação no conjunto de teste



E a inferência:

python

for batch in test_loader:

  X, y = batch

  # Inferência

  y_pred = modelo(X)



Para monitorar e salvar os checkpoints do modelo durante o treinamento, podemos utilizar o PyTorch Lightning:

In [ ]:
python
# Imports
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint

# Classe LightningModule
class LitModel(LightningModule):

  def __init__(self):
    super().__init__()
    self.modelo = nn.Sequential(...)

  def forward(self, x):
    return self.modelo(x)

  def training_step(self, batch, batch_idx):
    # Lógica de treinamento
    return loss

  def test_step(self, batch, batch_idx):
    # Lógica de teste/validação
    return loss

  def configure_optimizers(self):
    return optimizer

# Criando o modelo
modelo = LitModel()

# Callback de checkpoint
checkpoint_callback = ModelCheckpoint()

# Treinador
treinador = pl.Trainer(callbacks=[checkpoint_callback])

# Treinando
treinador.fit(modelo, train_loader, test_loader)




O PyTorch Lightning abstrai e automatiza vários aspectos do treinamento, como cálculo do número de épocas, avaliação no conjunto de teste e callbacks.

Também podemos utilizar o Weights & Biases (W&B) para monitoramento em tempo real:

In [ ]:
python
# Imports
import wandb
from pytorch_lightning.loggers import WandbLogger

wandb.init(project="meu_projeto")

treinador = Trainer(logger=WandbLogger())

# Durante treinamento
wandb.log({"loss": loss, "accuracy": acc})


O W&B integra com o Lightning para gerar dashboards com gráficos, permitindo acompanhar métricas como loss, accuracy, learning rate ao longo do treinamento.

Para deploy do modelo treinado, podemos utilizar o ONNX:

In [ ]:
python
# Exportando para .onnx
torch.onnx.export(modelo, dummy_input, "modelo.onnx")

# Inferência com ONNX Runtime
import onnxruntime as rt
sess = rt.InferenceSession("modelo.onnx")

entrada = ... # dados de entrada
saida = sess.run(None, {sess.get_inputs()[0].name: entrada})


Regularização - Evita overfitting:

In [ ]:
python



# Dropout
nn.Dropout(p=0.5)

# BatchNorm
nn.BatchNorm1d(num_features)

# Weight Decay
# Penaliza pesos elevados
optimizer = optim.Adam(modelo.parameters(), weight_decay=1e-5)


Augmentação de dados - Aumenta a variabilidade do conjunto de treinamento:

In [ ]:
python



# Imports
from torchvision import transforms

# Transformações aleatórias nos dados
transforms = transforms.Compose([
  transforms.RandomHorizontalFlip(),
  transforms.RandomRotation(degrees=15),
  transforms.ColorJitter(hue=0.1),
])

dataset = MeuDataset(X, y, transforms)

Hyperparameter Tuning - Encontra a melhor configuração:

In [ ]:
python



# Imports
from ray import tune

# Função para treinar modelo
def treinar_modelo(config):

  modelo = Net(config)
  modelo.fit(...)
  acc = modelo.score(...)

  return acc

# Procura exaustiva de hyperparameters
best_config = tune.run(
  treinar_modelo,
  config={
    "lr": tune.grid_search([0.01, 0.001, 0.0001]),
    "momentum": tune.grid_search([0.9, 0.99]),
    "batch_size": 32
  }
)

print(best_config)

Transfer Learning - Usa pesos pré-treinados:

In [ ]:
python



# Carregando modelo pré-treinado
modelo = models.resnet18(pretrained=True)

# Congelando parâmetros já treinados
for param in modelo.parameters():
  param.requires_grad = False

# Adicionando camadas novas para fine-tuning
modelo.fc = nn.Linear(512, num_classes)

Ensemble Models - Combina vários modelos:

In [ ]:
python



# Criando dois modelos
modeloA = Net()
modeloB = Net()

# Treinando
modeloA.fit(...)
modeloB.fit(...)

# Predições em conjunto
def ensemble_predict(x):
  return modeloA(x) + modeloB(x) / 2

Para servir o modelo treinado via API REST usando Django:

In [ ]:
python



# Imports
import torch
from django.conf.urls import url
from django.http import JsonResponse
from django.apps import AppConfig

class PredicaoConfig(AppConfig):
    name = 'predicao'

    def ready(self):
        # Carrega modelo
        global modelo
        modelo = Net()
        modelo.load_state_dict(torch.load('pesos.pth'))

# Rota para requisição POST
def model_predict(request):

    # Lê dados do corpo da requisição
    dados = json.loads(request.body)

    # Pré-processa dados
    dados = normalizar(dados)

    # Converte para tensor
    dados = torch.from_numpy(dados)

    # Prediz
    saida = modelo(dados)

    # Converte resposta para JSON
    resposta = saida.tolist()

    return JsonResponse(resposta, safe=False)

# URLs mapeadas para a view
urlpatterns = [
    url(r'^predict/$', model_predict)
]

E no arquivo urls.py principal do projeto Django:

In [ ]:
python



from django.urls import path, include

urlpatterns = [
    # Outras rotas
    path('api/', include('predicao.urls'))
]

Dessa forma integramos o modelo PyTorch em produção via API REST usando o framework Django.

O pré-processamento dos sinais fNIRS pode envolver técnicas como filtragem, normalização e extração de features:

In [ ]:
python



# Filtragem
from scipy.signal import butter, filtfilt

def filter_signal(signal):
  b, a = butter(order, cutoff, btype='lowpass')
  return filtfilt(b, a, signal)

# Normalização
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
signal = scaler.fit_transform(signal)

# Extração de features
from tsfresh import extract_features

X = extract_features(signal, column_id='id')

Para o classificador Random Forest:

In [ ]:
python



from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

# Treinamento
rf.fit(X_treino, y_treino)

# Predição
y_pred = rf.predict(X_teste)

# Avaliação
rf_accuracy = accuracy_score(y_teste, y_pred)

E para a CNN:

In [ ]:
python



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense

cnn = Sequential()
cnn.add(Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(n_timesteps,1)))
cnn.add(Dense(1, activation='sigmoid'))

# Treinamento
cnn.fit(X_treino, y_treino, epochs=10)

# Predição
y_pred = cnn.predict(X_teste)

# Avaliação
cnn_accuracy = accuracy_score(y_teste, y_pred>0.5)

Podemos comparar as métricas:

In [ ]:
python



print(f"Random Forest Accuracy: {rf_accuracy*100:.2f}%")
print(f"CNN Accuracy: {cnn_accuracy*100:.2f}%")